In [6]:
import os
import pandas as pd
import nibabel as nb
import numpy as np
import itertools
from nibabel import Nifti1Image
from nilearn.image import clean_img
from nilearn.input_data import NiftiMasker
from multiprocessing import Pool

from functions import process_voxel

In [7]:
data_root = '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2'
fmriprep_dir = os.path.join(data_root, 'derivatives','fmriprep23')
output_func_dir = os.path.join(data_root, 'derivatives', 'mvpa_04_musicnoise_bold', 'func_clean')
output_feat_stab_dir = os.path.join(data_root, 'derivatives', 'mvpa_08_stability_musicnoise_bold', 'feat_stab_full')
output_stab_dir = os.path.join(data_root, 'derivatives', 'mvpa_09_stabilityperfold_bold', 'stab_full')

In [8]:
subject = '18'

In [9]:
print(f"Estimating stability for subject {subject}...")

# Find the files
stab_feat_files = [os.path.join(output_feat_stab_dir, f) for f in os.listdir(output_feat_stab_dir) if f.endswith('_stab_features.npy') and f.startswith(f'sub-{subject}')]
stab_feat_files.sort()
print(stab_feat_files)
print(len(stab_feat_files))

Estimating stability for subject 18...
['/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2/derivatives/mvpa_08_stability_musicnoise_bold/feat_stab_full/sub-18_ses-01_task-02a_run-1_stab_features.npy', '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2/derivatives/mvpa_08_stability_musicnoise_bold/feat_stab_full/sub-18_ses-01_task-02a_run-3_stab_features.npy', '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2/derivatives/mvpa_08_stability_musicnoise_bold/feat_stab_full/sub-18_ses-01_task-02a_run-4_stab_features.npy']
3


In [10]:
# generate the folds
# test sample is run 1, 2, 3, 4, in this order, for folds 1, 2, 3, 4
 
n_folds = 4
if subject == '18': n_folds = 3

for ff in range(0,n_folds):

    print('--'*10)
    print(f'Fold {ff}...')
    stab_feat_files_fold = stab_feat_files.copy()
    del stab_feat_files_fold[ff]
    print(stab_feat_files_fold)

    # Concatenating the runs
    stab_feat = np.concatenate([np.load(f) for f in stab_feat_files_fold], axis=-1)

    print(f'Number of trials per condition per subject: {stab_feat.shape[-1]}')

    # Generate a list of indexes
    indexes = np.arange(stab_feat.shape[-1])
    combinations = np.array(list(itertools.combinations(indexes, 2)))
    n_combinations = len(combinations)
    print(f'Number of pairwise combinations: {n_combinations}')

    # Initialize STAB array
    STAB = np.zeros((stab_feat.shape[0], stab_feat.shape[1], stab_feat.shape[2]))

    # Use multiprocessing to process each x-coordinate slice in parallel
    with Pool(processes=8) as pool:
        results = pool.starmap(process_voxel, [(i, stab_feat, combinations) for i in range(stab_feat.shape[0])])

    # Collect the results
    for i, STAB_slice in results:
        STAB[i, :, :] = STAB_slice

    # Save STAB
    np.save(os.path.join(output_stab_dir, f'sub-{subject}_STAB_fold_{ff}.npy'), STAB)

    print(f"Done estimating stability for subject {subject}.")

--------------------
Fold 0...
['/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2/derivatives/mvpa_08_stability_musicnoise_bold/feat_stab_full/sub-18_ses-01_task-02a_run-3_stab_features.npy', '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2/derivatives/mvpa_08_stability_musicnoise_bold/feat_stab_full/sub-18_ses-01_task-02a_run-4_stab_features.npy']
Number of trials per condition per subject: 4
Number of pairwise combinations: 6
X coordinate 0/97...
X coordinate 1/97...
X coordinate 2/97...
X coordinate 3/97...
X coordinate 4/97...
X coordinate 5/97...
X coordinate 6/97...
X coordinate 7/97...
X coordinate 8/97...
X coordinate 9/97...
X coordinate 10/97...
X coordinate 11/97...
X coordinate 12/97...
X coordinate 13/97...
X coordinate 14/97...
X coordinate 15/97...
X coordinate 16/97...
X coordinate 17/97...
X coordinate 18/97...
X coordinate 19/97...
X coordinate 20/97...
X coordinate 21/97...
X coordinate 22/97...
X coordinate 24/97...


/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


X coordinate 23/97...
X coordinate 25/97...


/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


X coordinate 26/97...
X coordinate 28/97...


...

X coordinate 37/97...
X coordinate 38/97...
X coordinate 39/97...
X coordinate 40/97...


/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


X coordinate 41/97...
X coordinate 42/97...
X coordinate 43/97...
X coordinate 44/97...
X coordinate 45/97...
X coordinate 46/97...
X coordinate 47/97...
X coordinate 48/97...


/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


X coordinate 49/97...
X coordinate 50/97...
X coordinate 51/97...
X coordinate 52/97...
X coordinate 53/97...
X coordinate 54/97...
X coordinate 55/97...
X coordinate 56/97...
X coordinate 57/97...
X coordinate 58/97...
X coordinate 59/97...
X coordinate 60/97...
X coordinate 61/97...
X coordinate 62/97...
X coordinate 63/97...
X coordinate 64/97...
X coordinate 65/97...
X coordinate 66/97...
X coordinate 67/97...
X coordinate 68/97...
X coordinate 69/97...
X coordinate 70/97...
X coordinate 71/97...
X coordinate 72/97...
X coordinate 73/97...
X coordinate 74/97...
X coordinate 75/97...
X coordinate 76/97...
X coordinate 77/97...
X coordinate 78/97...
X coordinate 79/97...
X coordinate 80/97...
X coordinate 81/97...
X coordinate 82/97...
X coordinate 83/97...
X coordinate 84/97...
X coordinate 85/97...
X coordinate 86/97...
X coordinate 87/97...
X coordinate 88/97...
X coordinate 89/97...
X coordinate 90/97...
X coordinate 91/97...
X coordinate 92/97...
X coordinate 93/97...
X coordina